# 5 Quantum Algorithms

First, let's begin with simple, classical algorithms.

- **Standard Model**: a standard model of computation is a simple input (like a string of bits) and output system that can be used to describe any computation. The most common abstract model is the Turing machine, which is a theoretical construct that helps us understand the limits of what can be computed.


   <details>
   <summary>🧠 Details for nerds: Turing Machines.</summary>

   The Turing machine consists of an infinite tape divided into discrete cells and a read/write head that can move left or right to scan one of these cells at a time. It also consists of a finite set of states, a transition function that determines how the machine moves based on the current state and the symbol read from the tape, and a set of accepting states that indicate when the computation is complete. 

   Despite its simplicity, any algorithm you can run on a modern computer can also be encoded as a Turing machine program.
   </details>


- **Query model**: the query model of computation is a more abstract model that focuses on the interaction between a computation and an oracle. An oracle is a black box that can answer specific questions about a function, allowing the computation to access information without explicitly calculating it. 

    For example, the **OR** oracle takes an input $f: \Sigma^n \rightarrow \Sigma^m $ and produces an output $f(x) = 1$ if any of the inputs are 1, and $f(x) = 0$ otherwise.

   <details>
   <summary>🧠 Details for nerds: Query Model.</summary>

   In the query model, computations are allowed to make queries to an oracle, which can be thought of as a function $f: \Sigma^n \rightarrow \Sigma^m $ that takes an input and returns an output, where $n$ and $m$ are positive integers and $\Sigma = \{0, 1\}$. The oracle can be used to access information about the function without having to compute it directly. To say that a computation makes a query means that it evaluates the function $f$ once, and the result $f(x)$ is made available to the computation. We measure the efficiency of a computation in terms of the **number of queries** it makes to the oracle, which is often referred to as the query complexity of the computation.

   - It is worth noting that when we measure efficiency of query algorithms, we completely ignore the cost of implementing the oracle. This is because the oracle is assumed to be a black box, and we are only interested in how many queries it takes to solve a problem.
   </details>

  


